Importing Library

In [1]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Preprocessing

Converting Image Data for Training

In [3]:
train = []
class_id = []
current_dir = os.getcwd()

train_raw = pd.read_csv("traffic_train.csv", index_col = False)

def read_image_data(classes):

    for i in range(classes):
        path = os.path.join(current_dir, "Train", str(i)) # acess Train folder
        images = os.listdir(path) # get path for all images in the class
        for item in images:
            image = Image.open(os.path.join(path, item))
            image = image.resize((80,80))
            image = np.array(image)
            train.append(image) # image can be stored as array only
            class_id.append(i) # train file is in reverse order, so...

read_image_data(43)

train = np.array(train)
class_id = np.array(class_id)


Model Selection

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, class_id, test_size = 0.2, random_state = 31)

from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes = 43)
y_test = to_categorical(y_test, num_classes = 43) #to be used with the loss = categorical_crossentropy

In [17]:
print(X_train.shape)
X_train.shape[1:]

(31367, 80, 80, 3)


(80, 80, 3)

Training

In [23]:
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
import keras

model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (5, 5), activation = "relu", input_shape = (80, 80, 3))) # input layer

model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = "relu"))
model.add(MaxPool2D(pool_size = (2, 2))) # reduce spatial dimention
model.add(Dropout(rate = 0.3))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = "relu"))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = "relu"))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.3))
model.add(Flatten()) # converts to 1D to match Dense
model.add(Dense(units = 128, activation = "relu"))
model.add(Dropout(rate = 0.3))

model.add(Dense(units = 43, activation = "softmax")) # output layer

opt = keras.optimizers.Adam(learning_rate = 0.001, weight_decay = 1e-6)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics = ["accuracy"]) # soft probability
model.fit(X_train, y_train, batch_size = 10, epochs = 15, validation_data = (X_test, y_test))

/Users/jiyeon/anaconda3/envs/datascience/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/15
3137/3137 ━━━━━━━━━━━━━━━━━━━━ 691s 219ms/step - accuracy: 0.5481 - loss: 2.2480 - val_accuracy: 0.9555 - val_loss: 0.1537
Epoch 2/15
3137/3137 ━━━━━━━━━━━━━━━━━━━━ 697s 205ms/step - accuracy: 0.9120 - loss: 0.3107 - val_accuracy: 0.9639 - val_loss: 0.1260
Epoch 3/15
3137/3137 ━━━━━━━━━━━━━━━━━━━━ 600s 191ms/step - accuracy: 0.9320 - loss: 0.2370 - val_accuracy: 0.9719 - val_loss: 0.0976
Epoch 4/15
3137/3137 ━━━━━━━━━━━━━━━━━━━━ 606s 193ms/step - accuracy: 0.9398 - loss: 0.2092 - val_accuracy: 0.9712 - val_loss: 0.1074
Epoch 5/15
3137/3137 ━━━━━━━━━━━━━━━━━━━━ 621s 198ms/step - accuracy: 0.9458 - loss: 0.1917 - val_accuracy: 0.9872 - val_loss: 0.0535
Epoch 6/15
3137/3137 ━━━━━━━━━━━━━━━━━━━━ 1803s 575ms/step - accuracy: 0.9386 - loss: 0.2260 - val_accuracy: 0.9829 - val_loss: 0.0707
Epoch 7/15
3137/3137 ━━━━━━━━━━━━━━━━━━━━ 596s 190ms/step - accuracy: 0.9550 - loss: 0.1608 - val_accuracy: 0.9563 - val_loss: 0.1860
Epoch 8/15
3137/3137 ━━━━━━━━━━━━━━━━━━━━ 546s 174ms/step - a

In [41]:
path_test = os.path.join(current_dir, "Test")
test_images = os.listdir(path_test)
test_images.sort()

def get_array(path_list, general_path):

    data = []
    for item in path_list:
        image = Image.open(os.path.join(general_path, item))
        image = image.resize((80,80))
        image = np.array(image)
        data.append(image)

    return data

data = get_array(test_images, path_test)
test = np.array(data)
print(test.shape)

(12630, 80, 80, 3)


Testing

In [44]:
Y_prediction = model.predict(test)
output = np.argmax(Y_prediction, axis = 1)

test_csv = pd.read_csv("traffic_test.csv", index_col = 0)
ids = test_csv.index.to_list()
result = pd.DataFrame({"id": ids, "class": output})

395/395 ━━━━━━━━━━━━━━━━━━━━ 39s 98ms/step


Storing Result for Submission

In [ ]:
result.to_csv("NN_output.csv", index = False)